# Graphs and Networks

In [2]:
import nltk
import re
import networkx as nx
import gravis as gv
import numpy as np
import pandas as pd
import duckdb
import json

from nltk.tag import pos_tag, UnigramTagger, AffixTagger, BigramTagger, HiddenMarkovModelTagger
from nltk.stem import WordNetLemmatizer

from itertools import combinations
from collections import Counter

from urllib.request import urlopen
import matplotlib.pyplot as plt

## Istanbul Public Transportation Network (Full)

In [3]:
# ibb = pd.read_csv('https://data.ibb.gov.tr/dataset/a6855ce7-4092-40a5-82b5-34cf3c7e36e3/resource/db5d0618-05fc-417a-9bbe-16afda910832/download/hourly_transportation_202310.csv')
# ibb.to_parquet('../shared/data/ibb_2023-11_public_transportation.parquet')
ibb = pd.read_parquet('../shared/data/ibb_2023-11_public_transportation.parquet')
ibb

,transition_date,transition_hour,transport_type_id,road_type,line,transfer_type,number_of_passage,number_of_passenger
0,2023-10-01,0,2,RAYLI,LEVENT-HISAR USTU,Aktarma,3,3
1,2023-10-01,0,3,DENİZ,BESIKTAS-USKUDAR,Normal,10,7
2,2023-10-01,0,1,OTOYOL,KIRAC-AVCILAR METROBUS,Normal,2,2
3,2023-10-01,0,1,OTOYOL,HACI OSMAN METRO - KILYOS,Normal,1,1
4,2023-10-01,0,1,OTOYOL,GUNESLI - BEYAZIT,Normal,13,13
...,...,...,...,...,...,...,...,...
4178707,2023-10-31,23,1,OTOYOL,SARIYER-BESIKTAS,Normal,2,2
4178708,2023-10-31,23,1,OTOYOL,UGURMUMCU-ESENKENT METRO,Aktarma,1,1
4178709,2023-10-31,23,1,OTOYOL,SAHINTEPE-OLIMPIYATKOY METRO,Normal,4,4
4178710,2023-10-31,23,1,OTOYOL,ATAKENT-AKSARAY,Normal,7,7


In [4]:
lines = duckdb.sql("""
    select line, sum(number_of_passenger) as passengers from ibb
       group by line
       order by passengers desc
""").to_df()
lines

,line,passengers
0,AVCILAR-ZINCIRLIKUYU,21372447.0
1,HALKALI - GEBZE,18866811.0
2,YENIKAPI - HACIOSMAN,14878153.0
3,YENIKAPI - HAVALIMANI,10844715.0
4,KABATAS-BAGCILAR,10570686.0
...,...,...
858,SEYRANTEPE-KAGITHANE METRO,2.0
859,IC HATLAR,2.0
860,ALIBEYKOY METRO - ZINCIRLIKUYU METROBUS,2.0
861,YENIKAPI-ADALAR,2.0


In [5]:
edges = []
for x in lines.itertuples():
    tmp = [y.strip() for y in re.split('-|–|/',x[1])]
    if len(tmp) == 2:
        edges.append(tuple(tmp))
    elif len(tmp)>2:
        for aa in combinations(tmp,2):
            edges.append(tuple(aa))
edges

[('AVCILAR', 'ZINCIRLIKUYU'),
 ('HALKALI', 'GEBZE'),
 ('YENIKAPI', 'HACIOSMAN'),
 ('YENIKAPI', 'HAVALIMANI'),
 ('KABATAS', 'BAGCILAR'),
 ('KADIKOY', 'KARTAL'),
 ('USKUDAR', 'CEKMEKOY'),
 ('KABATAS', 'MAHMUTBEY'),
 ('TOPKAPI', 'HABIBLER'),
 ('CEVIZLIBAG', 'SOGUTLUCESME'),
 ('KIRAZLI', 'BASAKSEHIR'),
 ('KIRAZLI', 'METROKENT'),
 ('BASAKSEHIR', 'METROKENT'),
 ('EMINONU', 'ALIBEYKOY'),
 ('BOSTANCI', 'DUDULLU'),
 ('TUZLA', 'TOPKAPI'),
 ('KADIKOY', 'FINDIKLI MAHALLESI'),
 ('KADIKOY', 'YEDITEPE UNIVERSITESI'),
 ('FINDIKLI MAHALLESI', 'YEDITEPE UNIVERSITESI'),
 ('ALTUNIZADE', 'SULTANBEYLI'),
 ('UMRANIYE DEV.HAST', 'CAKMAK MAH'),
 ('UMRANIYE DEV.HAST', 'ATASEHIR'),
 ('UMRANIYE DEV.HAST', 'USTBOSTANCI'),
 ('CAKMAK MAH', 'ATASEHIR'),
 ('CAKMAK MAH', 'USTBOSTANCI'),
 ('ATASEHIR', 'USTBOSTANCI'),
 ('GUNESLI', 'BEYAZIT'),
 ('SULTANBEYLI(M.SINAN)', 'ADLIYE'),
 ('SULTANBEYLI(M.SINAN)', 'KARTAL DEV.HST'),
 ('ADLIYE', 'KARTAL DEV.HST'),
 ('KADIKOY', 'SARIGAZI'),
 ('KADIKOY', 'YENIDOGAN'),
 ('SARIGAZI', '

In [6]:
G = nx.from_edgelist(set(edges))
gv.vis(G)

In [7]:
sorted(nx.centrality.load_centrality(G).items(), key=lambda item: -item[1])

[('KADIKOY', 0.23091941429728557),
 ('TAKSIM', 0.13292843668750673),
 ('MECIDIYEKOY', 0.12416383059969933),
 ('EMINONU', 0.12269385765191974),
 ('USKUDAR', 0.0995486106695941),
 ('KARTAL', 0.09502381127521668),
 ('BAKIRKOY', 0.07334254759128077),
 ('KABATAS', 0.06421529982995379),
 ('SISLI', 0.06279596630051973),
 ('ALIBEYKOY METRO', 0.05434322232385149),
 ('SULTANBEYLI', 0.043896366103756794),
 ('KAVACIK', 0.04070077914108938),
 ('BESIKTAS', 0.03942402170889088),
 ('UMRANIYE', 0.034713743965591036),
 ('YENIBOSNA METRO', 0.03180331859300179),
 ('AVCILAR METROBUS', 0.03139789850646185),
 ('YENIKAPI', 0.030779248642367),
 ('YESILKENT', 0.029359017639674145),
 ('BAHCESEHIR', 0.02814187670140245),
 ('SARIYER', 0.02712557221971796),
 ('ARNAVUTKOY', 0.02474035221492075),
 ('BEYAZIT', 0.023151885231140184),
 ('YENIDOGAN', 0.02100593929985188),
 ('KARTAL METRO', 0.02035075284328205),
 ('CEVIZLI', 0.020344212806165055),
 ('CAGDASKENT', 0.019688586981548996),
 ('TUZLA', 0.019013399651721455),
 (

## Wikipedia Links

The data is taken from [SNAP](https://snap.stanford.edu/data/wikispeedia.html).

In [8]:
mernis = pd.read_sql_table('/home/kaygun/local/data/mernis/data_dump.sql')
mernis

TypeError: read_sql_table() missing 1 required positional argument: 'con'

## Character Network in Anna Karenina

In [9]:
nltk.download('averaged_perceptron_tagger', download_dir='/home/kaygun/local/lib/nltk_data')
nltk.download('wordnet', download_dir='/home/kaygun/local/lib/nltk_data')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kaygun/local/lib/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/kaygun/local/lib/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
def get_text(URL,begin,end):
    with urlopen(URL) as url:
        raw = url.read()
    text = raw.decode('utf-8')[begin:end]
    sentences = nltk.sent_tokenize(text)
    return sentences

In [11]:
# ulysses = get_text('https://www.gutenberg.org/ebooks/4300.txt.utf-8',1044,1553300)
anna_karenina = get_text('https://www.gutenberg.org/ebooks/1399.txt.utf-8',990,2005400)

In [12]:
def get_edges(sentences):
    res = []
    for x in sentences:
        words = nltk.word_tokenize(re.sub('_',' ',x))
        tagged = pos_tag(words)
        tmp = []
        for w in tagged:
            if w[1] == 'NNP' and len(w[0])>2 and w[0]!='Mrs' and w[0]!='Miss' and w[0]!='Ill':
                tmp.append(w[0])
        if len(tmp)==2:
            res.append(tuple(tmp))
        elif len(tmp)>2:
            for aa in combinations(tmp,2):
                res.append(tuple(aa))
    return res

In [13]:
anna_edges = get_edges(anna_karenina)
anna_edges

[('Prince', 'Stepan'),
 ('Prince', 'Arkadyevitch'),
 ('Prince', 'Oblonsky—Stiva'),
 ('Stepan', 'Arkadyevitch'),
 ('Stepan', 'Oblonsky—Stiva'),
 ('Arkadyevitch', 'Oblonsky—Stiva'),
 ('Alabin', 'Darmstadt'),
 ('Alabin', 'Darmstadt'),
 ('Darmstadt', 'Darmstadt'),
 ('Darmstadt', 'America'),
 ('Alabin', '—not'),
 ('Stepan', 'Arkadyevitch'),
 ('Stepan', 'Arkadyevitch'),
 ('Stepan', 'Arkadyevitch'),
 ('Stepan', 'Arkadyevitch'),
 ('Stepan', 'Arkadyevitch'),
 ('Stepan', 'Arkadyevitch'),
 ('Stepan', 'Arkadyevitch'),
 ('Stepan', 'Arkadyevitch'),
 ('Matvey', 'Stepan'),
 ('Matvey', 'Arkadyevitch'),
 ('Matvey', 'Matvey'),
 ('Stepan', 'Arkadyevitch'),
 ('Stepan', 'Matvey'),
 ('Arkadyevitch', 'Matvey'),
 ('Stepan', 'Arkadyevitch'),
 ('Stepan', 'Arkadyevitch'),
 ('Stepan', 'Matvey'),
 ('Arkadyevitch', 'Matvey'),
 ('Matvey', 'Anna'),
 ('Matvey', 'Arkadyevna'),
 ('Anna', 'Arkadyevna'),
 ('Thank', 'God'),
 ('Thank', 'Matvey'),
 ('Thank', 'Anna'),
 ('Thank', 'Arkadyevna'),
 ('God', 'Matvey'),
 ('God', 'Ann

In [14]:
graph = Counter(anna_edges)
edges = [ k for k,v in graph.items() if k[0]!= k[1] and v>10 ]
edges

[('Stepan', 'Arkadyevitch'),
 ('Anna', 'Arkadyevna'),
 ('Thank', 'God'),
 ('Darya', 'Alexandrovna'),
 ('Yes', 'Stepan'),
 ('Yes', 'Arkadyevitch'),
 ('Darya', 'Anna'),
 ('Anna', 'Alexey'),
 ('Anna', 'Alexandrovitch'),
 ('Alexey', 'Alexandrovitch'),
 ('Levin', 'Oblonsky'),
 ('Oblonsky', 'Levin'),
 ('Levin', 'Moscow'),
 ('Stepan', 'Levin'),
 ('Arkadyevitch', 'Levin'),
 ('Konstantin', 'Dmitrievitch'),
 ('Konstantin', 'Levin'),
 ('Levin', 'Sergey'),
 ('Levin', 'Ivanovitch'),
 ('Sergey', 'Ivanovitch'),
 ('Well', 'Levin'),
 ('Levin', 'Kitty'),
 ('Kitty', 'Dolly'),
 ('Levin', 'Shtcherbatskys'),
 ('Kitty', 'Levin'),
 ('Moscow', 'Levin'),
 ('Sergey', 'Levin'),
 ('Ivanovitch', 'Levin'),
 ('Nikolay', 'Levin'),
 ('Yes', 'Levin'),
 ('Yes', 'Kitty'),
 ('Levin', 'Stepan'),
 ('Levin', 'Arkadyevitch'),
 ('Chapter', 'Levin'),
 ('....', 'Stepan'),
 ('....', 'Arkadyevitch'),
 ('....', 'Levin'),
 ('Vronsky', 'Levin'),
 ('Vronsky', 'Petersburg'),
 ('Count', 'Vronsky'),
 ('Come', 'Levin'),
 ('Kitty', 'Vronsky

In [15]:
G = nx.from_edgelist(edges)
gv.vis(G,node_hover_neighborhood=True)

In [16]:
sorted(nx.centrality.betweenness_centrality(G).items(), key=lambda item: -item[1])

[('Levin', 0.3950966203742358),
 ('Anna', 0.17712792110804967),
 ('Kitty', 0.15262493853960835),
 ('Vronsky', 0.1497810618500274),
 ('Princess', 0.08240794856808883),
 ('Alexey', 0.05885215180948669),
 ('Alexandrovitch', 0.051268171402595715),
 ('....', 0.0482670488807251),
 ('Konstantin', 0.028053769725306835),
 ('Countess', 0.028053769725306835),
 ('Madame', 0.028053769725306835),
 ('Agafea', 0.028053769725306835),
 ('Varenka', 0.028053769725306835),
 ('Veslovsky', 0.028053769725306835),
 ('Stepan', 0.016581781748351003),
 ('Arkadyevitch', 0.016581781748351003),
 ('Betsy', 0.01597506331579972),
 ('Yes', 0.014027812566678727),
 ('Darya', 0.01064540369040661),
 ('Alexandrovna', 0.01064540369040661),
 ('Arkadyevna', 0.0),
 ('Thank', 0.0),
 ('God', 0.0),
 ('Oblonsky', 0.0),
 ('Moscow', 0.0),
 ('Dmitrievitch', 0.0),
 ('Sergey', 0.0),
 ('Ivanovitch', 0.0),
 ('Well', 0.0),
 ('Dolly', 0.0),
 ('Shtcherbatskys', 0.0),
 ('Nikolay', 0.0),
 ('Chapter', 0.0),
 ('Petersburg', 0.0),
 ('Count', 0.0),

## NBA Nettworks

In [18]:
# nba = pd.read_csv('https://sports-statistics.com/database/basketball-data/nba/2019-20_pbp.csv')
# nba.to_parquet('../shared/data/nba_2019-2020.parquet')
nba = pd.read_parquet('../shared/data/nba_2019-2020.parquet')
nba

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,...,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,Unnamed: 40
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,...,None,None,None,None,None,None,D. Favors - favorde01,M. Gasol - gasolma01,L. Ball - balllo01,NaN
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,...,None,None,None,None,None,None,None,None,None,NaN
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,...,None,None,None,None,None,None,None,None,None,NaN
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,...,None,None,None,None,None,None,None,None,None,NaN
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,None,...,None,None,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539260,/boxscores/202010110MIA.html,playoff,The Arena Bay Lake Florida,October 11 2020,7:30 PM,LAL,4,29,LAL,None,...,None,None,None,None,None,None,None,None,None,NaN
539261,/boxscores/202010110MIA.html,playoff,The Arena Bay Lake Florida,October 11 2020,7:30 PM,LAL,4,20,LAL,D. Howard makes 3-pt jump shot from 30 ft (ass...,...,None,None,None,None,None,None,None,None,None,NaN
539262,/boxscores/202010110MIA.html,playoff,The Arena Bay Lake Florida,October 11 2020,7:30 PM,LAL,4,15,LAL,None,...,None,None,None,None,None,None,None,None,None,NaN
539263,/boxscores/202010110MIA.html,playoff,The Arena Bay Lake Florida,October 11 2020,7:30 PM,LAL,4,0,LAL,End of 4th quarter,...,None,None,None,None,None,None,None,None,None,NaN


In [19]:
nba.columns

Index(['URL', 'GameType', 'Location', 'Date', 'Time', 'WinningTeam', 'Quarter',
       'SecLeft', 'AwayTeam', 'AwayPlay', 'AwayScore', 'HomeTeam', 'HomePlay',
       'HomeScore', 'Shooter', 'ShotType', 'ShotOutcome', 'ShotDist',
       'Assister', 'Blocker', 'FoulType', 'Fouler', 'Fouled', 'Rebounder',
       'ReboundType', 'ViolationPlayer', 'ViolationType', 'TimeoutTeam',
       'FreeThrowShooter', 'FreeThrowOutcome', 'FreeThrowNum', 'EnterGame',
       'LeaveGame', 'TurnoverPlayer', 'TurnoverType', 'TurnoverCause',
       'TurnoverCauser', 'JumpballAwayPlayer', 'JumpballHomePlayer',
       'JumpballPoss', 'Unnamed: 40'],
      dtype='object')

In [20]:
res = duckdb.sql("""
  select count(GameType) as count, Fouler, Fouled 
       from nba
       where GameType = 'regular'
       group by Fouler, Fouled
       order by count desc
""").to_df()
res

,count,Fouler,Fouled
0,455831,None,None
1,15,D. DeRozan - derozde01,None
2,14,R. Westbrook - westbru01,None
3,14,D. Green - greendr01,None
4,13,P. Tucker - tuckepj01,K. Leonard - leonaka01
...,...,...,...
31459,1,K. Kuzma - kuzmaky01,O. Anunoby - anunoog01
31460,1,M. Conley - conlemi01,J. Tatum - tatumja01
31461,1,A. Davis - davisan02,L. Dort - dortlu01
31462,1,D. Wright - wrighde01,D. Šarić - saricda01


In [21]:
edges = [(r[1],r[2]) for i,r in res.iterrows() if r[0]>5 and r[0]<100 and r[2]!=None]
edges

/tmp/ipykernel_132329/4114125294.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edges = [(r[1],r[2]) for i,r in res.iterrows() if r[0]>5 and r[0]<100 and r[2]!=None]


[('P. Tucker - tuckepj01', 'K. Leonard - leonaka01'),
 ('P. Beverley - beverpa01', 'J. Harden - hardeja01'),
 ('M. Harkless - harklma01', 'J. Harden - hardeja01'),
 ('W. Cauley-Stein - caulewi01', 'R. Gobert - goberru01'),
 ('R. Gobert - goberru01', 'N. Jokić - jokicni01'),
 ("R. O'Neale - onealro01", 'B. Ingram - ingrabr01'),
 ('L. Markkanen - markkla01', 'G. Antetokounmpo - antetgi01'),
 ('D. Brooks - brookdi01', 'D. Booker - bookede01'),
 ('M. Turner - turnemy01', 'J. Embiid - embiijo01'),
 ('D. Dedmon - dedmode01', 'J. Embiid - embiijo01'),
 ('R. Holmes - holmeri01', 'H. Whiteside - whiteha01'),
 ('D. Brooks - brookdi01', 'D. Fox - foxde01'),
 ('C. Joseph - josepco01', 'D. Lillard - lillada01'),
 ('J. Valančiūnas - valanjo01', 'R. Gobert - goberru01'),
 ('D. Finney-Smith - finnedo01', 'J. Harden - hardeja01'),
 ('J. Embiid - embiijo01', 'G. Antetokounmpo - antetgi01'),
 ('K. Bazemore - bazemke01', 'L. Dončić - doncilu01'),
 ('C. Osman - osmande01', 'M. Morris - morrima03'),
 ('I. Z

In [22]:
graph = nx.from_edgelist(edges)
gv.vis(graph, node_hover_neighborhood=True)

In [23]:
sorted(nx.centrality.betweenness_centrality(graph).items(), key=lambda item: -item[1])

[('D. Booker - bookede01', 0.20010268220895625),
 ('T. Young - youngtr01', 0.1765484408890299),
 ('G. Antetokounmpo - antetgi01', 0.17405009274407482),
 ('L. Dončić - doncilu01', 0.16291923683088846),
 ('N. Jokić - jokicni01', 0.16069703877002212),
 ('J. Butler - butleji01', 0.1412287776244242),
 ('B. Beal - bealbr01', 0.10741776362775085),
 ('D. Brooks - brookdi01', 0.09624582275030415),
 ('A. Davis - davisan02', 0.09358014300395869),
 ('M. Turner - turnemy01', 0.08922355919795104),
 ('J. Harden - hardeja01', 0.07792665436711536),
 ('R. Gobert - goberru01', 0.07135855324075555),
 ('S. Dinwiddie - dinwisp01', 0.06857532139222278),
 ('J. Embiid - embiijo01', 0.06674684683647555),
 ('A. Drummond - drumman01', 0.06665412052735997),
 ('D. Finney-Smith - finnedo01', 0.05970025848386921),
 ('G. Harris - harriga01', 0.056818938637120474),
 ('M. Thybulle - thybuma01', 0.05681893863712047),
 ('A. Gordon - gordoaa01', 0.05622470075094914),
 ('L. James - jamesle01', 0.05359373874098586),
 ('D. De

In [24]:
res = duckdb.sql("""
  select count(GameType) as count, Shooter, Assister 
       from nba
       where GameType = 'regular'
       group by Shooter, Assister
       order by count desc
""").to_df().dropna()
edges = [ (r[1],r[2]) for I,r in res.iterrows() if r[0] > 10 ]
edges

/tmp/ipykernel_132329/3469140451.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edges = [ (r[1],r[2]) for I,r in res.iterrows() if r[0] > 10 ]


[('A. Davis - davisan02', 'L. James - jamesle01'),
 ('H. Whiteside - whiteha01', 'D. Lillard - lillada01'),
 ('J. Murray - murraja01', 'N. Jokić - jokicni01'),
 ('M. Harrell - harremo01', 'L. Williams - willilo02'),
 ('N. Vučević - vucevni01', 'M. Fultz - fultzma01'),
 ('J. Collins - collijo01', 'T. Young - youngtr01'),
 ('T. Harris - harrito02', 'B. Simmons - simmobe01'),
 ('N. Jokić - jokicni01', 'J. Murray - murraja01'),
 ('S. Ibaka - ibakase01', 'K. Lowry - lowryky01'),
 ('T. Warren - warretj01', 'M. Brogdon - brogdma01'),
 ('K. Porziņģis - porzikr01', 'L. Dončić - doncilu01'),
 ('D. Booker - bookede01', 'R. Rubio - rubiori01'),
 ('C. McCollum - mccolcj01', 'D. Lillard - lillada01'),
 ('J. Valančiūnas - valanjo01', 'J. Morant - moranja01'),
 ('Z. LaVine - lavinza01', 'T. Satoranský - satorto01'),
 ('R. Gobert - goberru01', 'J. Ingles - inglejo01'),
 ('D. Robinson - robindu01', 'B. Adebayo - adebaba01'),
 ('D. Brooks - brookdi01', 'J. Morant - moranja01'),
 ('R. Westbrook - westbru0

In [25]:
gv.vis(nx.from_edgelist(edges))

In [26]:
G = nx.from_edgelist(edges)
sorted(nx.centrality.betweenness_centrality(G).items(), key=lambda item: -item[1])

[('J. Teague - teaguje01', 0.06802286097740642),
 ('J. McLaughlin - mclaujo01', 0.06102755830028562),
 ('R. Covington - covinro01', 0.058997946497946506),
 ('T. Young - youngtr01', 0.05467638422183878),
 ('S. Napier - napiesh01', 0.04878692015055651),
 ('J. Johnson - johnsja01', 0.0468622286804105),
 ('D. Dedmon - dedmode01', 0.0468622286804105),
 ('G. Dragić - dragigo01', 0.04604990968627332),
 ('K. Towns - townska01', 0.04104087831360553),
 ('J. Green - greenje02', 0.040262767535494806),
 ('D. Bertāns - bertada01', 0.03440246622064804),
 ('B. Bogdanović - bogdabo01', 0.02970320924866384),
 ('M. Beasley - beaslma01', 0.029644799644799637),
 ('R. Westbrook - westbru01', 0.026751026751026752),
 ('J. Crowder - crowdja01', 0.025615798343071074),
 ('C. Chiozza - chiozch01', 0.025610752883480158),
 ('J. Harden - hardeja01', 0.024450297177569903),
 ('T. Luwawu-Cabarrot - luwawti01', 0.023650591832410015),
 ('J. Ingles - inglejo01', 0.022747959111595474),
 ('J. Clarkson - clarkjo01', 0.021658

In [27]:
best_shooters = duckdb.sql("""
 select *, (make+miss) as total, make/total as percentage from 
    (pivot nba on ShotOutcome
         using count(ShotOutcome)
         group by Shooter
    )
    where total > 50
    order by percentage desc
""").to_df().dropna()

best_shooters.head(20)

,Shooter,make,miss,total,percentage
0,M. Robinson - robinmi01,253,88,341,0.741935
1,R. Williams - williro04,87,32,119,0.731092
2,D. Howard - howardw01,241,93,334,0.721557
3,D. Gafford - gaffoda01,94,40,134,0.701493
4,R. Gobert - goberru01,434,197,631,0.687797
5,D. Jones - jonesda03,121,57,178,0.679775
6,J. Hayes - hayesja02,174,85,259,0.671815
7,N. Noel - noelne01,192,94,286,0.671329
8,D. Jordan - jordade01,191,96,287,0.665505
9,I. Hartenstein - harteis01,44,23,67,0.656716


In [28]:
best_free_throwers = duckdb.sql("""
 select *, (make+miss) as total, make/total as percentage from 
    (pivot nba on FreeThrowOutcome
         using count(FreeThrowOutcome)
         group by FreeThrowShooter
    )
    where total > 25
    order by percentage desc
""").to_df().dropna()

best_free_throwers.head(20)

,FreeThrowShooter,make,miss,total,percentage
0,S. Curry - curryst01,26,0,26,1.000000
1,T. Snell - snellto01,32,0,32,1.000000
2,K. Martin - martike03,30,1,31,0.967742
3,B. Goodwin - goodwbr01,28,2,30,0.933333
4,M. Guduric - gudurma01,24,2,26,0.923077
5,K. Irving - irvinky01,94,8,102,0.921569
6,B. Wanamaker - wanambr01,140,12,152,0.921053
7,D. Booker - bookede01,468,41,509,0.919450
8,G. Mathews - mathega01,31,3,34,0.911765
9,D. Robinson - robindu01,100,10,110,0.909091
